<a href="https://colab.research.google.com/github/felixzhao/title_catgories_classification/blob/main/Job_title_classification_V2.2_encoding.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In this work, 
I need to encoding the title and feature.

1. encoding title which without preprocess
2. encoding title feature which has been processed, need to remove duplidate

# install dependency

In [27]:
! pip install sentence-transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


# install dependency

In [28]:
import numpy as np
import pandas as pd
from nltk import word_tokenize
from nltk.corpus import stopwords
import string
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer

In [29]:
# need to install stopwords and some other package by following code
# input "d" to download view, then input package name to start package download
import nltk
nltk.download("stopwords")
nltk.download("punkt")
nltk.download("wordnet")
nltk.download("omw-1.4")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [30]:
from sentence_transformers import SentenceTransformer, util

# load data

load data
This work loads data from google drive. Please put the files into "trademe_data" folder under the root of google drive.

Then run the following code to mount the google drive in colab.

You can follow the guide. https://towardsdatascience.com/different-ways-to-connect-google-drive-to-a-google-colab-notebook-pt-1-de03433d2f7a

In [31]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [32]:
root_path = "drive/MyDrive/trademe_data/"

In [33]:
raw_df = pd.read_csv(f"{root_path}raw.csv")
print(len(raw_df))
raw_df.head(1)

20000


,Unnamed: 0,x_title,y_cat_id,cat_1,cat_2,cat_3,x_title_feature,cats
0,0,unqualified asbestos remover,5192,trades & services,labourers,labourers,unqualified asbestos remover,trades & services-labourers-labourers


# functions

In [34]:
model = SentenceTransformer("all-MiniLM-L6-v2")

In [35]:
def get_embedding(text):
    if not text:
        print("Content is empty.")
        return None
    return model.encode(input, convert_to_tensor=True)

# Preprocess

In [36]:
raw_title_encoding_path =f"{root_path}raw_title_encoding.csv"
raw_title_feature_encoding_path =f"{root_path}raw_title_feature_encoding.csv"

# Title encoding

In [37]:
title_raw = raw_df.copy()[['x_title','y_cat_id','cat_1','cat_2','cat_3','cats']]
title_raw.head(2)

,x_title,y_cat_id,cat_1,cat_2,cat_3,cats
0,unqualified asbestos remover,5192,trades & services,labourers,labourers,trades & services-labourers-labourers
1,senior test analyst,5123,it,testing,testing,it-testing-testing


In [38]:
title_raw['ecoder_title'] = title_raw['x_title'].apply(lambda x: model.encode(x, convert_to_tensor=True))

In [50]:
print(len(title_raw))
title_raw.head(2)

20000


,x_title,y_cat_id,cat_1,cat_2,cat_3,cats,ecoder_title
0,unqualified asbestos remover,5192,trades & services,labourers,labourers,trades & services-labourers-labourers,"[tensor(-0.0663), tensor(0.0821), tensor(0.047..."
1,senior test analyst,5123,it,testing,testing,it-testing-testing,"[tensor(-0.0241), tensor(0.0556), tensor(-0.10..."


In [51]:
title_raw.to_csv(raw_title_encoding_path)

# Title Feature Encoding

In [42]:
title_feature_raw = raw_df.copy()[['x_title_feature','y_cat_id','cat_1','cat_2','cat_3','cats']]
print(len(title_feature_raw))
title_feature_raw.head(2)

20000


,x_title_feature,y_cat_id,cat_1,cat_2,cat_3,cats
0,unqualified asbestos remover,5192,trades & services,labourers,labourers,trades & services-labourers-labourers
1,senior test analyst,5123,it,testing,testing,it-testing-testing


In [43]:
title_feature_raw[title_feature_raw.duplicated()]

,x_title_feature,y_cat_id,cat_1,cat_2,cat_3,cats
72,support worker,5085,healthcare,community & social services,community & social services,healthcare-community & social services-communi...
94,chef,5100,hospitality & tourism,chefs,chefs,hospitality & tourism-chefs-chefs
160,labourer,5192,trades & services,labourers,labourers,trades & services-labourers-labourers
161,office administrator,5156,office & administration,administration,administration,office & administration-administration-adminis...
175,chef,5100,hospitality & tourism,chefs,chefs,hospitality & tourism-chefs-chefs
...,...,...,...,...,...,...
19983,labourers,5192,trades & services,labourers,labourers,trades & services-labourers-labourers
19985,frame truss fabricators,5186,trades & services,building & carpentry,building & carpentry,trades & services-building & carpentry-buildin...
19991,retail store manager,5176,retail,store managers,store managers,retail-store managers-store managers
19992,clinical psychologist,5094,healthcare,psychology & counselling,psychology & counselling,healthcare-psychology & counselling-psychology...


In [44]:
title_feature_raw = title_feature_raw.drop_duplicates()
print(len(title_feature_raw))

15205


In [45]:
title_feature_raw['ecoder_title_feature'] = title_feature_raw['x_title_feature'].apply(lambda x: model.encode(str(x), convert_to_tensor=True))

In [48]:
print(len(title_feature_raw))
title_feature_raw.head(2)

15205


,x_title_feature,y_cat_id,cat_1,cat_2,cat_3,cats,ecoder_title_feature
0,unqualified asbestos remover,5192,trades & services,labourers,labourers,trades & services-labourers-labourers,"[tensor(-0.0663), tensor(0.0821), tensor(0.047..."
1,senior test analyst,5123,it,testing,testing,it-testing-testing,"[tensor(-0.0241), tensor(0.0556), tensor(-0.10..."


In [49]:
title_feature_raw.to_csv(raw_title_feature_encoding_path)